In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from google.colab import drive


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [2]:
seed = 1
lr = 1e-3
momentum = 0.5
batch_size = 64
test_batch_size = 64
epochs = 5
no_cuda = False
log_interval = 100

In [3]:
# Model

class Net(nn.Module) :
    def __init__(self) :
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,20,5,1)
        self.conv2 = nn.Conv2d(20,50,5,1)
        self.fc1   = nn.Linear(4*4*50, 500)
        self.fc2   = nn.Linear(500,10)

    def forward(self, x) :
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2) 
        x = x.view(-1,4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

In [4]:
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


## Preprocess

In [19]:
google_path = '/content/MyDrive/MyDrive/fastcampus/DL_300/이미지분석/dataset/mnist_png'
# os.listdir('MyDrive/MyDrive/fastcampus/DL_300/이미지분석/dataset/mnist_png')
os.chdir(google_path)

In [20]:
os.listdir('training')

['3', '7', '9', '5', '8', '2', '4', '6', '0', '1']

In [22]:
os.listdir()[1]

'training'

In [23]:
train_dir = google_path + '/' + os.listdir()[1]
test_dir  = google_path + '/' + os.listdir()[0]

In [24]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [13]:
print(device)

cuda


In [25]:
train_dataset = datasets.ImageFolder(root = train_dir, # torch로 불러오면 gray scale이 되지 않기 떄문에 input에 채널을 3으로 바꿔준다.
                                     transform = transforms.Compose([
                                                        transforms.ToTensor(),
                                                        transforms.Normalize((0.1307,),(0.3081,))
                                     ]))    

In [26]:
test_dataset = datasets.ImageFolder(root = test_dir, # torch로 불러오면 gray scale이 되지 않기 떄문에 input에 채널을 3으로 바꿔준다.
                                     transform = transforms.Compose([
                                                        transforms.ToTensor(),
                                                        transforms.Normalize((0.1307,),(0.3081,))
                                     ]))  

In [27]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


Dataset ImageFolder
    Number of datapoints: 10000
    Root location: /content/MyDrive/MyDrive/fastcampus/DL_300/이미지분석/dataset/mnist_png/testing
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )